# ЗАДАЧА 1
Необходимо написать запрос, который выгрузит данные из БД о пользователях из Канады, при этом разделив их на три группы.

РЕШЕНИЕ:

    SELECT id, views,
    CASE
           WHEN views <100 THEN 3
           WHEN views >=100 AND views <350 THEN 2
           WHEN views>=350 THEN 1
    END as group
    FROM stackoverflow.users
    WHERE location LIKE '%Canada%' AND views > 0

# ЗАДАЧА 2

Дополнить предыдущий запрос так, чтобы были отображены лидеры групп. Лидерство определяется наибольшим числом просмотров в группе.

РЕШЕНИЕ:

    WITH user_table AS (
                        SELECT id,
                               views,
                               CASE
                                   WHEN views <100 THEN 3
                                   WHEN views >=100 AND views <350 THEN 2
                                   WHEN views>=350 THEN 1
                               END as user_group
                        FROM stackoverflow.users
                        WHERE location LIKE '%Canada%' AND views > 0
                        )
    SELECT id, user_group, views
    FROM user_table
    WHERE views IN (
                    SELECT max_views
                    FROM(
                            SELECT user_group, MAX(views) as max_views
                            FROM user_table
                            GROUP BY user_group) as t1
                    )
    ORDER BY views DESC, id

# ЗАДАЧА 3

Необходимо выяснить, какие пары товаров покупают вместе чаще всего.

Пары товаров должны быть представлены в виде списков из двух наименований. Пары товаров внутри списков должны быть отсортированы в порядке возрастания наименования. Результат отсортируйте сначала по убыванию частоты встречаемости пары товаров в заказах, затем по колонке pair — по возрастанию.

Сложность заключается в том, что товары внсены списком в поле product_ids, и их необходимо сначала выделить из списка.

РЕШЕНИЕ:

    WITH cte AS (
                 SELECT order_id,
                        name as product_name
                 FROM   (
                         SELECT order_id,
                                unnest(product_ids) AS ids
                         FROM   orders
                         WHERE  order_id NOT IN (SELECT order_id
                                                 FROM   user_actions
                                                 WHERE  action = 'cancel_order')
                         ) as sq1
                     LEFT JOIN products as p ON p.product_id = sq1.ids
                 ), 
         cte2 AS (
                    SELECT DISTINCT order_id,
                            array[t1.product_name, t2.product_name] as pair
                    FROM   cte as t1 join cte as t2 using(order_id)
                    WHERE  t1.order_id = t2.order_id
                    AND t1.product_name <> t2.product_name
                    AND t1.product_name < t2.product_name
                 )
        SELECT pair,
               count(*) as count_pair
        FROM   cte2
        GROUP BY pair
        ORDER BY count_pair desc, pair

# ЗАДАЧА 4
Найти скользящее среднее по количеству заказов за 3 предыдущих дня

    SELECT date,
           orders_count,
           round(avg(orders_count) OVER (ORDER BY date rows between 3 preceding and 1 preceding),
                 2) as moving_avg
    FROM   (SELECT creation_time::date as date,
                   count(order_id) as orders_count
            FROM   orders
            WHERE  order_id not in (SELECT order_id
                                    FROM   user_actions
                                    WHERE  action = 'cancel_order')
            GROUP BY date
            ORDER BY date) as t